In [11]:
import requests
import json
import pandas as pd
import re
import nltk

def df_scan_and_clean(df_path):
    url = "https://www.virustotal.com/api/v3/files"

    files = { "file": (df_path, open(df_path, "rb"), "text/csv") }
    headers = {
        "accept": "application/json",
        "X-Apikey": "f8be13e86eaa07aa9be829de24cfe7af333ed266d34d92b1b1e1d5a6d9476b5d"
    }

    response = requests.post(url, files=files, headers=headers)
    text_file = requests.get(json.loads(response.text)['data']['links']['self'], headers=headers).text

    statistics = json.loads(text_file)['data']["attributes"]['stats']

    if statistics['suspicious'] + statistics['malicious'] > 0:
        raise Exception("Malicious or suspicious content")
    else:
        #clean data
        df_to_send = clean_pii(df_path)
        #upload to IPFS

        print("Uploaded to IPFS")

In [25]:
nlp = spacy.load("en_core_web_sm")

def replace_last_names(text):
    doc = nlp(text)
    modified_text = text
    people = False
    
    for ent in doc.ents:
        
        if ent.label_ == "PERSON":
            
            name_parts = ent.text.split()
            if len(name_parts) > 1:  # Ensure there's 1+ last names
                people = True
                last_name = name_parts[-1]
                anonymized_name = name_parts[0] + " " + last_name[0] + "."
                modified_text = modified_text.replace(ent.text, anonymized_name)

    return people, modified_text


def pii_finder(text):
    if not isinstance(text, str):
        text = str(text)
        
    about_people = False
    patterns = {
                "email_address": r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b",
                "phone_number": r"\b(?:\+?\d{1,3}\s?)?\(?\d{3}\)?[-\s]?\d{3}[-\s]?\d{4}\b",
                "ssn": r"\b\d{3}[-]?\d{2}[-]?\d{4}\b",
                "ip_address": r"\b(?:(?:(?:25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])\.){3}(?:25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])|(?:[0-9A-Fa-f]{1,4}:){7}[0-9A-Fa-f]{1,4})\b",
                "last_name": r"\b([A-Z][a-z]+(?:\s[Jr\.|Sr\.|II|III|IV])?)\b",
                "credit_card": r"\b(?:\d{4}[-\s]?){3}\d{4}|\b(?:\d{4}[-\s]?){3}\d{3}\b|\b(?:\d{4}[-\s]?){2}\d{4}[-\s]?\d{4}\b|\b\d{13,19}\b",
                "age": r"\b(?:(?:age|around|approximately|about|was|were)\s*)?(1[01]?[0-9]|[1-9]?\d)(?:\s*years?\s*(?:\d+\s*days?)?\s*old|\s*yrs?\.?|old)?\b",
                "birth_date": r"\b(\d{4}[-/]\d{1,2}[-/]\d{1,2}|\d{1,2}[-/]\d{1,2}[-/]\d{4})\b"
            }
     
    for category, pattern in patterns.items():
        
        matches = re.finditer(pattern, text)
        
        if category == "phone_number" and any(matches):
            return "[REDACTED PHONE NUMBER]"
        
        elif category == "email_address" and any(matches):
            return "[REDACTED EMAIL]"
        
        elif category == "ssn" and any(matches):
            raise Exception("SSNs Included; please remove. Failure to do so again will result in a 0.5 ETH penalty.")
        
        elif category == "ip_address" and any(matches):
            raise Exception("IP Addresses Included; please remove. Failure to do so again will result in a 0.5 ETH penalty.")
        
        elif category == 'credit_card' and any(matches):
            raise Exception("Credit Card Information Detected; please remove. Failure to do so again will result in a 0.5 ETH penalty.")
        
        elif category == 'last_name':            
            about_people, cleaned = replace_last_names(text)
            return cleaned
        
        elif category == 'age' and about_people and any(matches):
            return "[REDACTED AGE]"
        
        elif category == 'birth_date' and about_people and any(matches):
            return "[REDACTED BIRTH DATE]"
        
        else:
            return text

In [35]:
def clean_pii(df_link):
    offenses = 0
    df = pd.read_csv(df_link)
    cast_type = True
    for column in df.columns:
        try:
            df[column] = df[column].apply(pii_finder)
        except Exception as e: 
            offenses += 1
            cast_type = False
            print(f"Error processing column '{column}': {e}")
            break 
    
    if offenses > 3:
        place_holder = 'place_holder'
        #find way to tage wallets with illegal disclosures
    
    if cast_type:
        for column in df.columns:
            try:
                df[column] = pd.to_numeric(df[column], errors='coerce')
            except ValueError:
                try:
                    df[column] = pd.to_datetime(df[column], errors='coerce')
                except ValueError:
                    if df[column].str.lower().isin(['true', 'false']).all():
                        df[column] = df[column].str.lower() == 'true'
                    else:
                        pass
    return df